In [ ]:
#!The one working

Mounted at /content/gdrive


In [1]:
# import tensorflow as tf
import numpy as np
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import torch
import nltk
from flask import Flask, request, render_template, redirect, url_for
from werkzeug.utils import secure_filename

In [2]:
# !pip install pdfplumber
# !pip install nltk
# !pip install -U gensim
# !pip install flask-ngrok
# !pip install werkzeug
# !pip install numpy

In [11]:
import os

In [2]:
import pdfplumber

def pdf_extract(file_name):
    directory = "C:/Users/MohdS/OneDrive/Desktop/Major-Project/Flask-QA-System/docs"
    pdf_txt = ""
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if(filename == file_name):
            pdf_txt = '' # Reset pdf_txt for each file
            with pdfplumber.open(os.path.join(directory, filename)) as pdf:  # Use os.path.join for path concatenation
                for pdf_page in pdf.pages:
                    single_page_text = pdf_page.extract_text()
                    pdf_txt = pdf_txt + single_page_text
    return pdf_txt


In [ ]:
extracted_text = pdf_extract("AMZN-Q3-2020-Earnings-Release.pdf")  # Replace "example.pdf" with the name of your PDF file
print(extracted_text)


In [9]:
print(pdf_extract('fdi.pdf'))

In [ ]:
#New Way pdf Extractor
#!pip install pdf_layout_scanner

In [ ]:
# from pdf_layout_scanner import layout_scanner
# toc=layout_scanner.get_toc('docs/test.pdf')
# pages=layout_scanner.get_pages('docs/test.pdf')
# print(len(pages))

# for i in range(len(pages)):
#     print(pages[i])

In [3]:
def clean_sentence(sentence, stopwords=False):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  if stopwords:
    sentence = remove_stopwords(sentence)
  return sentence

def get_cleaned_sentences(tokens, stopwords=False):
  cleaned_sentences = []
  for row in tokens:
    cleaned = clean_sentence(row, stopwords)
    cleaned_sentences.append(cleaned)
  return cleaned_sentences

In [4]:
def retrieveAndPrintFAQAnswer(question_embedding, sentence_embeddings, sentences):
  max_sim = -1
  index_sim = -1
  for index, embedding in enumerate(sentence_embeddings):
    sim = cosine_similarity(embedding, question_embedding)[0][0]
    if sim > max_sim:
      max_sim = sim
      index_sim = index
  
  return index_sim

===================================================================================================================

===================================================================================================================

In [5]:
# !pip install transformers==3.1.0
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

c:\Users\MohdS\OneDrive\Desktop\Major-Project\Flask\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:

# from transformers import BertForQuestionAnswering, BertTokenizer

# # Initialize and load the BERT model
# model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# # Initialize and load the BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# # Save the model and tokenizer
# joblib.dump(model, 'bert_model.pkl')
# joblib.dump(tokenizer, 'bert_tokenizer.pkl')


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['bert_tokenizer.pkl']

In [6]:
model = joblib.load('bert_model.pkl')
tokenizer = joblib.load('bert_tokenizer.pkl')


c:\Users\MohdS\OneDrive\Desktop\Major-Project\Flask\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
question = "Outcome of the make in india initiative?"
answer = bert_drive('fdi.pdf', question)
print(answer)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MohdS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors


NameError: name 'bert_qa' is not defined

In [8]:
def expand_split_sentences(pdf_txt):
  
  nltk.download('punkt')
  new_chunks = nltk.sent_tokenize(pdf_txt)
  length = len(new_chunks)
  #for i in range(length):
    #tmp_token = tokenizer.encode(new_chunks[i])
    #print('The input has a total of {:} tokens.'.format(len(tmp_token)))

  new_df = [];
  for i in range(length):
    paragraph = ""
    for j in range(i, length):
      #tmp_str = paragraph + new_chunks[j]
      tmp_token = tokenizer.encode(paragraph + new_chunks[j])
      length_token = len(tmp_token)
      if length_token < 510:
        #print(length_token)
        paragraph = paragraph + new_chunks[j]
      else:
        #print(length_token)
        break;
    #print(len(tokenizer.encode(paragraph)))
    new_df.append(paragraph)
  return new_df
  #for i in new_df:
    #print(i)

In [9]:


def bert_drive(file_name, question):
    import numpy
    text = pdf_extract(file_name)
    max_score = 0
    final_answer = ""
    new_df = expand_split_sentences(text)
    tokens = []
    s_scores = numpy.array([])
    e_scores = numpy.array([])

    for new_context in new_df:
        ans, score, start_score, end_score, token = bert_qa(question, new_context)
        if score > max_score:
            max_score = score
            # Convert scalar values to PyTorch tensors
            start_score_tensor = torch.tensor(start_score)
            end_score_tensor = torch.tensor(end_score)
            s_scores = start_score_tensor.detach().numpy()
            e_scores = end_score_tensor.detach().numpy()
            tokens = token
            final_answer = ans

    return final_answer, s_scores, e_scores, tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MohdS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have be

(96, array(0.75, dtype=float32), array(1.75, dtype=float32), 'india jumped to 63rd place in world bank ’ s ease of doing business ranking as per world bank ’ s doing business report ( dbr ) 2020 . this is driven by reforms in the areas of starting a business , paying taxes , trading across borders , and resolving insolvency . recently , government has taken various steps in addition to ongoing schemes to boost domestic and foreign investments in india . these include the national infrastructure pipeline , reduction in corporate tax , easing liquidity problems of nbfcs and banks , policy measures to boost domestic manufacturing . government of india has also promoted domestic manufacturing of goods through public procurement orders , phased manufacturing programme ( pmp ) , schemes for production linked incentives of various ministries . further , with a view to support , facilitate and provide investor friendly ecosystem to investors investing in india , the union cabinet on 03rd june 

In [ ]:
# question = "What were the various steps taken by the government to boost domestic and foreign investment in India ?"

# answer = (bert_drive('fdi.pdf', question))
# print(answer)


In [52]:
from flask import Flask, request, render_template, redirect, url_for
from werkzeug.utils import secure_filename

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if request.form.get('btn') == 'index':
            if 'upload' not in request.files:
                return "No file selected!"
            upload = request.files['upload']
            if upload.filename == '':
                return "No file selected!"
            file_name = secure_filename(upload.filename)
            path = "docs"
            import os
            if not os.path.exists('docs'):
                os.makedirs('docs')
            upload.save(os.path.join(path, secure_filename(upload.filename)))
            return redirect(url_for('qa', file_name=file_name))
        elif request.form.get('btn') == 'qa':
            question = request.form.get('question')
            file_name = request.form.get('file_name')
            if file_name is None:
                return "No file selected!"
            answer, s_scores, e_scores, tokens = bert_drive(file_name, question)
            return render_template('qa.html', answer=answer, question=question, file_name=file_name)
    return render_template('index.html')

@app.route('/upload/', methods=['GET', 'POST'])
def upload():
    return render_template('upload.html')

@app.route('/qa/', methods=['GET', 'POST'])
def qa():
    file_name = request.args.get('file_name')
    if file_name is None:
        return "No file selected"
    file_names = [f for f in os.listdir("docs")]
    return render_template('qa.html', file_names=file_names, file_name=file_name)

# Add your bert_drive function here

In [ ]:
# !pip install pyngrok

In [16]:
!ngrok authtoken ENTER_YOUR_AUTH_TOKEN

Authtoken saved to configuration file: C:\Users\MohdS\AppData\Local/ngrok/ngrok.yml


In [53]:
app.run()

 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/May/2024 16:48:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2024 16:48:38] "GET /upload/?type=bert HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2024 16:48:42] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [06/May/2024 16:48:42] "GET /qa/?file_name=fdi.pdf HTTP/1.1" 200 -
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MohdS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[2024-05-06 16:49:05,355] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "c:\Users\MohdS\OneDrive\Desktop\Major-Project\Flask\lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\MohdS\OneDrive\Desktop\Major-Project\Flask\lib\site-packages\flask\app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\MohdS\OneDrive\Desktop\Major-Project\Flask\lib\site-packages\flask\app.